# Calculate historical departure rate estimates

This notebook loads historical quarterly federal payroll data, and uses it to estimate the employee departure rates using two types of identifiers — the "Pseudo ID" supplied by OPM until recently, and unique names — and then compares the results of these two appraoches.

In [1]:
import pandas as pd
from collections import OrderedDict
import sys
import os
from glob import glob

## Load data

In [2]:
paths_1973 = glob("../inputs/fedscope-data/1973-09-to-2014-06/non-dod/status/*.txt")

In [3]:
paths_2014 = glob("../inputs/fedscope-data/2014-09-to-2016-09/non-dod/status/*.txt")

In [4]:
paths_2016 = glob("../inputs/fedscope-data/2016-12-to-2017-03/non-dod/status/*.txt")

In [5]:
fwf_columns = OrderedDict([
    ('Pseudo ID', (0, 9)),
    ('Name', (9, 32)),
    ('File Date', (32, 38)),
    ('TOA', (83, 85)),
    ('NSFTP Indicator', (86, 87)),
])

In [6]:
def process_fwf(path):
    sys.stderr.write(path.split("/")[-1] + "\n")
    slices = fwf_columns.values()
    with open(path, encoding="latin-1") as f:
        rows = []
        for line in f:
            row = [ line[start:end].strip() for start, end in slices ]
            rows.append(row)
        df = pd.DataFrame(rows, columns=list(fwf_columns.keys()))\
            .assign(**{ 
                "Name Is Provided": lambda x: ~(x["Name"].str.contains(r"NAME WITHHELD|UNKNOWN"))
            })

    return df[
        (df["NSFTP Indicator"] == "1")
    ].drop_duplicates(subset=[ "Pseudo ID" ])[[
        "File Date", "Pseudo ID", "Name", "Name Is Provided", "TOA"
    ]]

In [7]:
cols_2014 = [
    "Last Name",
    "First Name",
    "File Date",
    "Agency",
    "SubAgency",
    "State",
    "Age Range",
    "YSD (Year Since Degree) Range",
    "Education Level",
    "Pay Plan",
    "Grade",
    "LOS (Length of Service) Level",
    "Occupation",
    "Occupational Category (PATCO)",
    "Adjusted Basic Pay",
    "Supervisor",
    "TOA",
    "Work Schedule",
    "NSFTP Indicator",
]

In [8]:
def process_csv(path, sep=";", names=None):
    sys.stderr.write(path.split("/")[-1] + "\n")
    df = pd.read_csv(
        path,
        sep=sep,
        low_memory=False,
        names=names,
    ).assign(**{
        "Name": lambda x: x[[ "Last Name", "First Name" ]].fillna("").apply(",".join, axis=1),
        "TOA": lambda x: x["TOA"].str.slice(0, 2),
    }).assign(**{
        "Name Is Provided": lambda x: ~(x["Name"].str.contains(r"NAME WITHHELD|NAME UNKNOWN"))
    })
    return df[
        (df["NSFTP Indicator"].astype(str).str.slice(0, 1) == "1")
    ][[
        "File Date", "Name", "Name Is Provided", "TOA"
    ]]

In [9]:
def calculate_turnover_rates(subset, q1, q2):
    q1_nonpolitical = q1[q1["TOA"] != "55"]
    if "Pseudo ID" in q1.columns and "Pseudo ID" in q2.columns:
        by_id = 1 - q1_nonpolitical["Pseudo ID"].isin(q2["Pseudo ID"]).mean()
    else:
        by_id = None
    
    q1_unique_names = q1_nonpolitical["Name"].value_counts()\
        .pipe(lambda x: x[x == 1]).index
    
    by_name = 1 - q1_unique_names.isin(q2["Name"].unique()).mean()

    return {
        "subset": subset,
        "q1": str(q1["File Date"].iloc[0])[:6],
        "q2": str(q2["File Date"].iloc[0])[:6],
        "q1_count": len(q1_nonpolitical),
        "q1_unique_name_count": len(q1_unique_names),
        "prop_missing_by_id": by_id,
        "prop_missing_by_name": by_name,
    }

In [10]:
process_1973 = lambda path: process_fwf(path)
process_2014 = lambda path: process_csv(path, sep="|", names=cols_2014)
process_2016 = lambda path: process_csv(path, sep=";")

In [11]:
dfgen_1973 = [ (process_1973, path) for path in paths_1973[-59:] ]
dfgen_2014 = [ (process_2014, path) for path in paths_2014 ]
dfgen_2016 = [ (process_2016, path) for path in paths_2016 ]

df_gens = dfgen_1973 + dfgen_2014 + dfgen_2016

In [12]:
comparisons = []

q1 = df_gens[0][0](df_gens[0][1])
for gen in df_gens[1:]:
    q2 = gen[0](gen[1])

    comparisons.append(calculate_turnover_rates(
        "overall",
        q1,
        q2,
    ))
    
    comparisons.append(calculate_turnover_rates(
        "named_only",
        q1[q1["Name Is Provided"]],
        q2[q2["Name Is Provided"]],
    ))

    del q1
    q1 = q2

Status_Non_DoD_1999_12.txt
Status_Non_DoD_2000_03.txt
Status_Non_DoD_2000_06.txt
Status_Non_DoD_2000_09.txt
Status_Non_DoD_2000_12.txt
Status_Non_DoD_2001_03.txt
Status_Non_DoD_2001_06.txt
Status_Non_DoD_2001_09.txt
Status_Non_DoD_2001_12.txt
Status_Non_DoD_2002_03.txt
Status_Non_DoD_2002_06.txt
Status_Non_DoD_2002_09.txt
Status_Non_DoD_2002_12.txt
Status_Non_DoD_2003_03.txt
Status_Non_DoD_2003_06.txt
Status_Non_DoD_2003_09.txt
Status_Non_DoD_2003_12.txt
Status_Non_DoD_2004_03.txt
Status_Non_DoD_2004_06.txt
Status_Non_DoD_2004_09.txt
Status_Non_DoD_2004_12.txt
Status_Non_DoD_2005_03.txt
Status_Non_DoD_2005_06.txt
Status_Non_DoD_2005_09.txt
Status_Non_DoD_2005_12.txt
Status_Non_DoD_2006_03.txt
Status_Non_DoD_2006_06.txt
Status_Non_DoD_2006_09.txt
Status_Non_DoD_2006_12.txt
Status_Non_DoD_2007_03.txt
Status_Non_DoD_2007_06.txt
Status_Non_DoD_2007_09.txt
Status_Non_DoD_2007_12.txt
Status_Non_DoD_2008_03.txt
Status_Non_DoD_2008_06.txt
Status_Non_DoD_2008_09.txt
Status_Non_DoD_2008_12.txt
S

In [13]:
results = pd.DataFrame(comparisons)[[
    "subset",
    "q1",
    "q2",
    "q1_count",
    "q1_unique_name_count",
    "prop_missing_by_id",
    "prop_missing_by_name",
]].sort_values([ "q1", "q2", "subset" ])

results["approach_diff"] = (results["prop_missing_by_name"] - results["prop_missing_by_id"]).abs()

In [14]:
results.head()

,subset,q1,q2,q1_count,q1_unique_name_count,prop_missing_by_id,prop_missing_by_name,approach_diff
1,named_only,199912,200003,580375,552723,0.021483,0.021448,0.000034
0,overall,199912,200003,908249,552723,0.020410,0.021448,0.001039
3,named_only,200003,200006,576846,549571,0.016354,0.016376,0.000022
2,overall,200003,200006,904361,549571,0.015730,0.016376,0.000646
5,named_only,200006,200009,578363,550963,0.021274,0.021352,0.000078


Note: In mid-2006, the Department of Justice changed the Pseudo IDs for a large number of unnamed employees. Below, we exclude that transitional quarter.

In [15]:
results[
    (results["subset"] == "overall") &
    (results["q1"] != "200606")
].assign()\
    .sort_values("approach_diff", ascending=False)\
    .dropna()\
    ["approach_diff"].round(5).describe()

count    57.000000
mean      0.000661
std       0.000626
min       0.000000
25%       0.000190
50%       0.000480
75%       0.000760
max       0.002610
Name: approach_diff, dtype: float64

In [16]:
results[
    (results["subset"] == "overall")
]["q1_unique_name_count"].describe()

count        69.000000
mean     618176.333333
std       47403.035346
min      549571.000000
25%      580351.000000
50%      600333.000000
75%      659870.000000
max      689551.000000
Name: q1_unique_name_count, dtype: float64

In [17]:
results.to_csv(
    "../outputs/historical-estimates.csv",
    index=False,
)

---

---

---